__Import Required Libraries__

In [ ]:
import cv2
import face_recognition
import numpy as np
import os
import PIL.Image

__Set up the Video Capture__

In [ ]:
# Initialize the webcam capture
video_capture = cv2.VideoCapture(0)  # 0 for default camera

# Check if the camera is successfully opened
if not video_capture.isOpened():
    print("Error: Camera could not be opened.")
else:
    print("Camera successfully opened.")


__Load the Image__

In [ ]:
def load_image_file(file, mode='RGB'):
    """
    Loads an image file (.jpg, .png, etc) into a numpy array

    :param file: image file name or file object to load
    :return: image contents as numpy array
    """
    im = PIL.Image.open(file)
    if mode:
        im = im.convert(mode)
    return np.array(im)

__Load and encord data__

In [ ]:

def load_known_faces(known_faces_dir):

    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(known_faces_dir):
        filepath = os.path.join(known_faces_dir, filename)
        try:
            image = load_image_file(filepath)
            encodings = face_recognition.face_encodings(image)
            if encodings:  # Ensure at least one encoding is found
                known_face_encodings.append(encodings[0])
                known_face_names.append(os.path.splitext(filename)[0])
        except Exception as e:
            print(f"Error loading file {filepath}: {e}")
    return known_face_encodings, known_face_names

path=r'C:\send_ml\final\media\profile_picture'
load_known_faces(path)


In [ ]:
def recognize_faces(frame, known_face_encodings, known_face_names):
    # Detect faces in the current frame
    face_locations = face_recognition.face_locations(frame, model='hog')
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    recognized_faces = []

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        
        recognized_faces.append((name, (left, top, right, bottom)))
    
    return recognized_faces


In [ ]:
def display_frame_with_recognition(known_face_encodings, known_face_names):
    while True:
        ret, frame = video_capture.read()
        if not ret:

            print("breaking")
            break

        # Resize for faster processing
        frame = cv2.resize(frame, (640, 480))

        # Recognize faces
        recognized_faces = recognize_faces(frame, known_face_encodings, known_face_names)

        # Draw bounding boxes and names on the frame
        for name, (left, top, right, bottom) in recognized_faces:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Display the frame with recognized faces
        cv2.imshow('Video Stream', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

known_faces_dir = r"C:\send_ml\final\media\profile_picture"

# Load known faces and names
known_face_encodings, known_face_names = load_known_faces(known_faces_dir)
print(known_face_names)

# Start displaying frame with face recognition
display_frame_with_recognition(known_face_encodings, known_face_names)


In [ ]:
# Release the video capture and close windows
video_capture.release()
cv2.destroyAllWindows()